In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.iolib.summary2 import summary_col
import itertools

In [2]:
# Read only total_tract_population column from acs_data.csv
tract_pop = pd.read_csv(
    "../data/acs_data.csv",
    usecols=["tract", "total_tract_population"],
    dtype={"total_tract_population": np.float32, "tract": np.int32},
)
tract_pop.columns

Index(['tract', 'total_tract_population'], dtype='object')

# Run Regression 

In [3]:
# Load the data
dtype_dict = {
    "B01003_001E_adj_supply_any_avg_tract": np.float32,
    "B01003_001E_adj_supply_hr_avg_tract": np.float32,
    "B01003_001E_totcost_any_avg_tract": np.float32,
    "state": np.int32,
    "county": np.int32,
    "tract": np.int32,
    "median_household_income": np.float32,
    "gini_index": np.float32,
    "educational_attainment": np.float32,
    "employment_status": np.float32,
    "housing_tenure": np.float32,
    "poverty_status": np.float32,
    "commute_time": np.float32,
    "without_health_insurance": np.float32,
    "white": np.float32,
    "black": np.float32,
    "am_indian_alaska_native": np.float32,
    "asian": np.float32,
    "other_race": np.float32,
    "hispanic_latino": np.float32,
    "density": np.float32,
}


data = pd.read_csv(
    "../data/acs_data_all.csv", dtype=dtype_dict
)  # Replace with the path to your data file

# Merge tract_pop to data
data = data.merge(tract_pop, how="left", on="tract", validate="1:1")

# Scale median_household_income by 10,000
data["median_household_income"] = data["median_household_income"] / 10000

# Scale density by 10,000
data["density"] = data["density"] / 10000

# Rename the variables
data.rename(
    columns={
        "B01003_001E_adj_supply_any_avg_tract": "Average Supply of Any Provider",
        "B01003_001E_adj_supply_hr_avg_tract": "Average Supply of HR Provider",
        "B01003_001E_totcost_any_avg_tract": "Average Total Cost of Any Provider",
        "median_household_income": "Median Household Income",
        "gini_index": "Gini Index",
        "educational_attainment": "Educational Attainment",
        "employment_status": "Employment Status",
        "housing_tenure": "Housing Tenure",
        "poverty_status": "Poverty Status",
        "commute_time": "Commute Time",
        "without_health_insurance": "Without Health Insurance",
        "white": "Share of White Population",
        "black": "Share of Black Population",
        "am_indian_alaska_native": "Share of American Indian/Alaska Native Population",
        "asian": "Share of Asian Population",
        "other_race": "Share of Other Race Population",
        "hispanic_latino": "Share of Hispanic/Latino Population",
        "density": "Population Density",
        "tract_pop": "total_tract_population",
    },
    inplace=True,
)

# calculate state population
state_population = data.groupby("state")["total_tract_population"].transform("sum")

# create tract_weight
data["tract_weight"] = data["total_tract_population"] / state_population

In [4]:
data.head()

,Average Supply of Any Provider,Average Supply of HR Provider,Average Total Cost of Any Provider,state,county,tract,Median Household Income,Gini Index,Educational Attainment,Employment Status,...,Without Health Insurance,Share of White Population,Share of Black Population,Share of American Indian/Alaska Native Population,Share of Asian Population,Share of Other Race Population,Share of Hispanic/Latino Population,Population Density,total_tract_population,tract_weight
0,0.024394,0.007030,140.768204,27,27001,1231966324,4.6548,0.4291,0.142012,0.455980,...,0.000000,0.000458,0.000042,0.000741,6.338997e-08,3.097119e-08,0.000150,0.000218,2240.0,0.000404
1,0.277115,0.000000,49.739838,27,27001,1231966424,4.8580,0.3815,0.084266,0.442195,...,0.008319,0.000446,0.000020,0.001586,4.847469e-09,4.922516e-08,0.000067,0.000226,2284.0,0.000411
2,0.481750,0.081783,147.445908,27,27001,1231966524,4.4543,0.4515,0.155082,0.498758,...,0.006657,0.000719,0.000079,0.000672,4.847469e-08,2.739537e-09,0.000240,0.004462,3483.0,0.000627
3,0.420769,0.028768,91.856232,27,27001,1231966624,4.8708,0.4339,0.121503,0.485145,...,0.006061,0.000593,0.000070,0.002534,0.000000e+00,2.518452e-09,0.000134,0.000295,2970.0,0.000535
4,0.009878,0.000000,59.393177,27,27001,1231986725,5.0110,0.4564,0.087353,0.438879,...,0.001050,0.000406,0.000011,0.000000,5.220351e-08,1.345738e-10,0.000033,0.000295,1905.0,0.000343


In [5]:
def run_regression_analysis(covariates_list, outcome_variable, data, model_names, scalar=1):
    regression_results = {}
    rmse_results = {}

    for covariates, model_name in zip(covariates_list, model_names):
        X = sm.add_constant(data[covariates])
        y = data[outcome_variable] * scalar

        model = sm.OLS(y, X)
        result = model.fit()

        rmse = np.sqrt(mean_squared_error(y, result.predict()))

        # Use model name as a key to store results
        regression_results[model_name] = result
        rmse_results[model_name] = rmse

    return regression_results, rmse_results

### Access to Any Provider

In [6]:
# Define the covariates

# First set of covariates will include racial group shares
first_set = [
    "Share of White Population",
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
]

# Second set of covariates will include the first set plus educational attainment, median income
second_set = first_set + [
    "Median Household Income",
    "Educational Attainment",
]

# Third set of covariates will include the second set plus age, population density, and unemployment rate
third_set = second_set + [
    "Gini Index",
    "Employment Status",
    "Housing Tenure",
    "Poverty Status",
    "Commute Time",
    "Without Health Insurance",
]

# The fourth set of covariates will include the third set plus the population density
fourth_set = third_set + ["Population Density"]

# Combine all covariates into a list of lists
covariates_list = [first_set, second_set, third_set, fourth_set]

# Define the outcome variable
outcome_variable = "Average Supply of Any Provider"

# Define the scalar
scalar = 1  # Replace with your desired scalar

# Define model names
model_names = ["First Set", "Second Set", "Third Set", "Fourth Set"]

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list, outcome_variable, data, model_names, scalar
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}"
    }
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [7]:
# Here I'm assuming covariate_list is the list containing the variable names in the order you want
covariate_list_ordered = [
    'Commute Time',
    'Educational Attainment',
    'Employment Status',
    'Gini Index',
    'Housing Tenure',
    'Median Household Income',
    'Population Density',
    'Poverty Status',
    'Share of American Indian/Alaska Native Population',
    'Share of Asian Population',
    'Share of Black Population',
    'Share of Hispanic/Latino Population',
    'Share of Other Race Population',
    'Share of White Population',
    'Without Health Insurance',
    'const',
]

# Separate list for non-standard error metrics
metrics_list = ['N', 'R2', 'Adjusted R2', 'RMSE']

# Last non-empty row name
last_non_empty = ''

# New index
new_index = []

for name in summary_df1.index:
    if name == '':
        new_index.append(last_non_empty + '_se')
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [item for sublist in [[var, var + "_se"] for var in covariate_list_ordered] for item in sublist if item in summary_df1.index]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith('_se')]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: '' for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Commute Time,,,0.124,0.124
,,,(0.199),(0.199)
Educational Attainment,,0.908***,0.342**,0.339**
,,(0.110),(0.143),(0.148)
Employment Status,,,0.537***,0.536***
,,,(0.131),(0.134)
Gini Index,,,0.243,0.243
,,,(0.183),(0.184)
Housing Tenure,,,-0.327***,-0.326***
,,,(0.077),(0.079)


### High Rated Providers

In [8]:
# Define the outcome variable
outcome_variable = "Average Supply of HR Provider"

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list, outcome_variable, data, model_names, scalar
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}"
    }
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [9]:
# Here I'm assuming covariate_list is the list containing the variable names in the order you want
covariate_list_ordered = [
    'Commute Time',
    'Educational Attainment',
    'Employment Status',
    'Gini Index',
    'Housing Tenure',
    'Median Household Income',
    'Population Density',
    'Poverty Status',
    'Share of American Indian/Alaska Native Population',
    'Share of Asian Population',
    'Share of Black Population',
    'Share of Hispanic/Latino Population',
    'Share of Other Race Population',
    'Share of White Population',
    'Without Health Insurance',
    'const',
]

# Separate list for non-standard error metrics
metrics_list = ['N', 'R2', 'Adjusted R2', 'RMSE']

# Last non-empty row name
last_non_empty = ''

# New index
new_index = []

for name in summary_df1.index:
    if name == '':
        new_index.append(last_non_empty + '_se')
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [item for sublist in [[var, var + "_se"] for var in covariate_list_ordered] for item in sublist if item in summary_df1.index]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith('_se')]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: '' for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Commute Time,,,0.153**,0.165**
,,,(0.068),(0.066)
Educational Attainment,,0.676***,0.480***,0.365***
,,(0.038),(0.049),(0.049)
Employment Status,,,-0.061,-0.141***
,,,(0.045),(0.044)
Gini Index,,,-0.034,-0.004
,,,(0.062),(0.061)
Housing Tenure,,,-0.208***,-0.156***
,,,(0.026),(0.026)


### Total Cost

In [10]:
# Define the outcome variable
outcome_variable = "Average Supply of HR Provider"

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list, outcome_variable, data, model_names, scalar
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}"
    }
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [11]:
# Here I'm assuming covariate_list is the list containing the variable names in the order you want
covariate_list_ordered = [
    'Commute Time',
    'Educational Attainment',
    'Employment Status',
    'Gini Index',
    'Housing Tenure',
    'Median Household Income',
    'Population Density',
    'Poverty Status',
    'Share of American Indian/Alaska Native Population',
    'Share of Asian Population',
    'Share of Black Population',
    'Share of Hispanic/Latino Population',
    'Share of Other Race Population',
    'Share of White Population',
    'Without Health Insurance',
    'const',
]

# Separate list for non-standard error metrics
metrics_list = ['N', 'R2', 'Adjusted R2', 'RMSE']

# Last non-empty row name
last_non_empty = ''

# New index
new_index = []

for name in summary_df1.index:
    if name == '':
        new_index.append(last_non_empty + '_se')
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [item for sublist in [[var, var + "_se"] for var in covariate_list_ordered] for item in sublist if item in summary_df1.index]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith('_se')]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: '' for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Commute Time,,,0.153**,0.165**
,,,(0.068),(0.066)
Educational Attainment,,0.676***,0.480***,0.365***
,,(0.038),(0.049),(0.049)
Employment Status,,,-0.061,-0.141***
,,,(0.045),(0.044)
Gini Index,,,-0.034,-0.004
,,,(0.062),(0.061)
Housing Tenure,,,-0.208***,-0.156***
,,,(0.026),(0.026)
